In [1]:
import os
import shutil
import tensorflow
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
from PIL import Image

MUSHROOMS_PATH = 'mushrooms_dataset'

# Directory for the images and its subdirectories
images_dir = os.path.join(MUSHROOMS_PATH, 'images_FasterRCNN', 'images_correct')
subdirs = [os.path.join(images_dir, subdir) for subdir in os.listdir(images_dir) if os.path.isdir(os.path.join(images_dir, subdir))]

In [2]:
#Now we have some ideas for dividing the dataset into training and testing sets. We can use the train_test_split function from scikit-learn to divide the dataset into training and testing sets.
#But for that we will have to put the every image into array and then into a dataframe
#Then we will have to use ImageDataGenerator and flow_from_dataframe to load the images from the dataframe

#Second idea is to manually create the test set by taking 20% of the images from each class and putting them into a separate directory.
#We will then use ImageDataGenerator and flow_from_directory to load the images from the directory.

#In both ideas we need to take in consider stratification, so that the distribution of classes in the training and testing sets is similar.
#For example, if in one class there are 10 images and in another one there are 8 images, we want both  of them to have the same percentage of images in the training and testing sets.

#Third idea is to use the splitfolders library to divide the dataset into training and testing sets.
#But again we have to stratify the dataset which is not supported by that library.

#So the first idea might require a lot of memory usage, the second idea needs us to well do this manually which is not very efficient.
#And the third idea is not supporting stratification.

#So for now we will use the first idea and divide the dataset into training and testing sets using the train_test_split function from scikit-learn which has the stratify parameter.


In [3]:
#So the process with the first idea is as follows:
#1. Load the images and its corresponding labels into a dataframe.
#2. Divide the dataset into training and testing sets using the train_test_split function from scikit-learn with stratification.
#3. Use ImageDataGenerator and flow_from_dataframe to load the images from the dataframe.

In [4]:
len(subdirs)

1002

In [5]:
data = []
for subdir in subdirs:
    label = os.path.basename(subdir) #we specify the label for each image
    for filename in os.listdir(subdir):
        if filename.endswith('.jpg'):
            data.append((os.path.join(subdir, filename), label)) #we need to include whole path of the image for using flow_from_dataframe because it reads the images directly from the file system using the paths provided in the DataFrame.
data_df = pd.DataFrame(data, columns=['filename', 'label'])

In [6]:
data_df.head()

,filename,label
0,mushrooms_dataset\images_FasterRCNN\images_cor...,Abortiporus_biennis
1,mushrooms_dataset\images_FasterRCNN\images_cor...,Abortiporus_biennis
2,mushrooms_dataset\images_FasterRCNN\images_cor...,Abortiporus_biennis
3,mushrooms_dataset\images_FasterRCNN\images_cor...,Abortiporus_biennis
4,mushrooms_dataset\images_FasterRCNN\images_cor...,Abortiporus_biennis


In [7]:
train_df, test_df = train_test_split(data_df, test_size=0.2, stratify=data_df['label'], random_state=42)

In [8]:
datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.2, shear_range=0.2, validation_split=0.25,) #we use 25% from the 80% of the training set as the validation set which will be the same amount as the testing set

train_data = datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filename',
    y_col='label',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32,
    subset='training'
)


val_data = datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filename',
    y_col='label',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32,
    subset='validation'
)

datagen_test = ImageDataGenerator(rescale=1./255)

test_data = datagen_test.flow_from_dataframe(
    dataframe=test_df,
    x_col='filename',
    y_col='label',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32
)

Found 64639 validated image filenames belonging to 1002 classes.
Found 21546 validated image filenames belonging to 1002 classes.
Found 21547 validated image filenames belonging to 1002 classes.


In [9]:
#---DONE---
#It worked, but why do we have only 6903 classes in the test set and 7504 in training and validation sets? 
# Perhaps there are not enough images in some classes???

# It is probably true beacause when we use stratify parameter in train_test_split function, it tries to keep the distribution of classes in the training and testing sets similar.
# But if there are not enough images in some classes, it will not be able to keep the distribution of classes similar in the training and testing sets.
# So we have few solutions to this
# 1. Ensure that each class has a minimum number of instances before splitting the data into training and testing sets - that worked!!!!
# 2. Use the stratify sampling only on the classes with sufficient instances, and randomly split the ones with too few instances

Could have - we could create our own model but since we have massive amount of images its easier to use pre-trained one

In [10]:
# Creating the first model
# model = Sequential()
# model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dense(7504, activation='softmax'))

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# history = model.fit(train_data, validation_data=val_data, epochs=20)



Since we will use couple of models to compare each ones results, its good to create a function for saving model

In [11]:
from tensorflow.saved_model import save
def saveModel(model, model_name):

    if not os.path.exists('models'):
        os.makedirs('models')
    model.save(f'models/{model_name}.h5') #for saving the model in h5 format
    model.export(f'models/{model_name}') #for saving the model in saved_model format

Training on InceptionV3 model

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
x = Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.1))(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)

predictions = Dense(1002, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.summary()

optimizer = Adam(learning_rate=0.0001)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1, min_lr=0.000001)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_data, validation_data=val_data, epochs=10, callbacks=[early_stopping])

In [13]:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

0 input_layer
1 conv2d
2 batch_normalization
3 activation
4 conv2d_1
5 batch_normalization_1
6 activation_1
7 conv2d_2
8 batch_normalization_2
9 activation_2
10 max_pooling2d
11 conv2d_3
12 batch_normalization_3
13 activation_3
14 conv2d_4
15 batch_normalization_4
16 activation_4
17 max_pooling2d_1
18 conv2d_8
19 batch_normalization_8
20 activation_8
21 conv2d_6
22 conv2d_9
23 batch_normalization_6
24 batch_normalization_9
25 activation_6
26 activation_9
27 average_pooling2d
28 conv2d_5
29 conv2d_7
30 conv2d_10
31 conv2d_11
32 batch_normalization_5
33 batch_normalization_7
34 batch_normalization_10
35 batch_normalization_11
36 activation_5
37 activation_7
38 activation_10
39 activation_11
40 mixed0
41 conv2d_15
42 batch_normalization_15
43 activation_15
44 conv2d_13
45 conv2d_16
46 batch_normalization_13
47 batch_normalization_16
48 activation_13
49 activation_16
50 average_pooling2d_1
51 conv2d_12
52 conv2d_14
53 conv2d_17
54 conv2d_18
55 batch_normalization_12
56 batch_normalization_

In [14]:
for layer in model.layers[:279]:
   layer.trainable = False
for layer in model.layers[279:]:
   layer.trainable = True

In [15]:
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
model.fit(train_data, validation_data=val_data, epochs=40, callbacks=[early_stopping, reduce_lr])

Epoch 1/40
2020/2020 ━━━━━━━━━━━━━━━━━━━━ 752s 369ms/step - accuracy: 0.2362 - loss: 5.3619 - val_accuracy: 0.3014 - val_loss: 4.8424 - learning_rate: 1.0000e-04
Epoch 2/40
2020/2020 ━━━━━━━━━━━━━━━━━━━━ 745s 368ms/step - accuracy: 0.3156 - loss: 4.6534 - val_accuracy: 0.3261 - val_loss: 4.6316 - learning_rate: 1.0000e-04
Epoch 3/40
2020/2020 ━━━━━━━━━━━━━━━━━━━━ 743s 367ms/step - accuracy: 0.3580 - loss: 4.3228 - val_accuracy: 0.3503 - val_loss: 4.4629 - learning_rate: 1.0000e-04
Epoch 4/40
2020/2020 ━━━━━━━━━━━━━━━━━━━━ 742s 367ms/step - accuracy: 0.4018 - loss: 4.0619 - val_accuracy: 0.3633 - val_loss: 4.3700 - learning_rate: 1.0000e-04
Epoch 5/40
2020/2020 ━━━━━━━━━━━━━━━━━━━━ 744s 368ms/step - accuracy: 0.4372 - loss: 3.8307 - val_accuracy: 0.3700 - val_loss: 4.3158 - learning_rate: 1.0000e-04
Epoch 6/40
2020/2020 ━━━━━━━━━━━━━━━━━━━━ 763s 377ms/step - accuracy: 0.4715 - loss: 3.6359 - val_accuracy: 0.3837 - val_loss: 4.2754 - learning_rate: 1.0000e-04
Epoch 7/40
2020/2020 ━━━━━━━

In [17]:
#Evaluate the model
model.evaluate(test_data)

674/674 ━━━━━━━━━━━━━━━━━━━━ 177s 263ms/step - accuracy: 0.4643 - loss: 2.9321


[2.9550905227661133, 0.46321994066238403]

In [18]:
#Save the model
saveModel(model, 'inception_v3_mushroomsv1_3_8')

INFO:tensorflow:Assets written to: models/inception_v3_mushroomsv1_3_8\assets


INFO:tensorflow:Assets written to: models/inception_v3_mushroomsv1_3_8\assets


Saved artifact at 'models/inception_v3_mushroomsv1_3_8'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1002), dtype=tf.float32, name=None)
Captures:
  1366813962896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1366813964240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1366813963856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1366813964048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1366813964624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1366813963280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1366813965200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1366813962704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1366813965584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1366814458512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136681445832

In [19]:
# model = tensorflow.keras.models.load_model('models/inception_v3_mushroomsv1_3_5.h5')
# model.export('models/inception_v3_mushroomsv1_3_5_SavedModel')

In [20]:
# Command for running the model with TensorFlow Serving
# $ docker pull tensorflow/serving:latest-gpu - for pulling the image of TensorFlow Serving
# $ docker run --rm -p 8501:8501 --name tfserving_inception -v "C:\Users\Adam\Desktop\FungEye\FungEye\FungEyeAi\models\inception_v3_mushroomsv1_3_5_SavedModel\1:/models/inception/1" -e MODEL_NAME=inception tensorflow/serving:latest-gpu

In [21]:
import json, requests

def predict_image(image_path):
    # preprocess the image
    image = Image.open(image_path)
    image = image.resize((224, 224))
    image = np.array(image) / 255.0
    image = image.reshape(1, 224, 224, 3)

    # specify the endpoint and make the request
    endpoint = 'http://localhost:8501/v1/models/inception:predict'
    headers = {'Content-Type': 'application/json'}
    batch_json = {'signature_name': 'serving_default', 'instances': image.tolist()} #we need to convert the image to a list because the model expects a list of instances

    response = requests.post(endpoint, json=batch_json, headers=headers)
    predictions = json.loads(response.text)['predictions']

    # lets make the predictions more readable, i have a list of class names in the mushroom_names.txt file and we can combine the class names with the predictions
    prediction_list = []
    with open('mushroom_names.txt', 'r') as file:
        class_names = file.read().splitlines()
        for i, prediction in enumerate(predictions[0]):
            prediction_list.append((class_names[i], prediction))

    # Sort the predictions by probability
    prediction_list.sort(key=lambda x: x[1], reverse=True)
    return prediction_list[:5]

# predict_image('mushrooms_dataset/images/Cryptoporus_volvatus/57468.jpg')